In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def preprocessing(fname):
    list_time = [] # time when fingers moved (str)
    with open(fname + '_time.txt') as f:
        for line in f:
            list_time.append(line.split('\t')[2].rstrip('\n'))
    list_seconds = [] # time when finger moved (float)
    for time in list_time:
        l = time.split(':')
        list_seconds.append(float(l[0]) * 3600 + float(l[1]) * 60 + float(l[2]))
    data = pd.read_csv(fname + '.txt', names=['time', 'ch1', 'ch2', 'ch3', 'ch4']) # wave data with time
    data['move'] = 0 # movement of finger
    for time in list_seconds: #match wave and the time fingers moved
        i = abs(data['time'] - time).idxmin()
        if fname[0] == 'R':
            data.at[data.index[i], 'move'] = 1
        else:
            data.at[data.index[i], 'move'] = -1
    data = data[(data['time'] > 30) & (data['time'] < data['time'].max() - 20)]
    data['inner'] = data['ch3'] - data['ch2']
    data['outer'] = data['ch4'] - data['ch1']
    data = data.drop(['ch1', 'ch2', 'ch3', 'ch4'], axis=1).reset_index(drop=True)
    n = len(data)
    for i in range(n - 1):
        if data.at[i + 1, 'time'] - data.at[i, 'time'] > 0.1:
            data = data.append((data.loc[i] + data.loc[i + 1]) / 2, ignore_index=True)
    data.loc[n:, 'move'] = 0
    data.sort_values(by='time', ascending=False)
    diff = pd.DataFrame(index=[], columns=['inner', 'outer'])
    for i in range(len(data) - 1):
        diff.loc[i] = data.loc[i + 1, ['inner', 'outer']] - data.loc[i, ['inner', 'outer']]
    diff['move'] = data.loc[:len(data) - 1, 'move']
    diff1= diff.loc[0:len(diff)-4].reset_index(drop=True)#divide into five
    diff1= diff1.rename(columns={'inner': 'inner1', 'outer': 'outer1'}).drop('move',axis=1)
    diff2= diff.loc[1:len(diff)-3].reset_index(drop=True)
    diff2 = diff2.rename(columns={'inner': 'inner2', 'outer': 'outer2'}).drop('move',axis=1)
    diff3 = diff.loc[2:len(diff)-2].reset_index(drop=True)
    diff3 = diff3.rename(columns={'inner': 'inner3', 'outer': 'outer3'})
    diff4 = diff.loc[3:len(diff)-1].reset_index(drop=True)
    diff4 = diff4.rename(columns={'inner': 'inner4', 'outer': 'outer4'}).drop('move',axis=1)
    diff5 = diff.loc[4:len(diff)].reset_index(drop=True)
    diff5 = diff5.rename(columns={'inner': 'inner5', 'outer': 'outer5'}).drop('move',axis=1)
    #merge
    diff_all = pd.concat([diff1, diff2,diff3,diff4,diff5], axis=1)
    #export as CSV file
    diff_all.to_csv (fname+'.csv')

In [ ]:
preprocessing('R1')

In [ ]:
preprocessing('L1')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
t1 = np.array(R1['time'][:-1])
t2 = np.array(R1['time'][1:])
x = t1
y = t2 - t1
plt.scatter(x, y)
plt.show()